<a href="https://colab.research.google.com/github/rhodes-byu/cs180-winter25/blob/main/notebooks/11-dbscan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DBSCAN: Density-Based Spatial Clustering of Applications with Noise

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs, make_circles, make_moons
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

## Blobs Data

In [ ]:
X, labels = make_blobs(n_samples = 500, centers = 4, random_state = 42, cluster_std = 1)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c = labels)

### Estimating Hyperparameters

In [ ]:
# Estimate k-nearest distances
dists = pairwise_distances(X)
dists.sort(axis = 1)

In [ ]:
# kth nearest distance
k_dists = dists[:, 10]
k_dists.sort()

In [ ]:
# Plot the sorted k-distances
plt.plot(k_dists)
plt.title('K-Nearest Distances')
plt.xlabel('Points')
plt.ylabel('Distance')
plt.show()

### Fitting DBSCAN

In [ ]:
# Fit DBSCAN
db = DBSCAN(eps = .5, min_samples = 10).fit(X)
cluster_labels = db.labels_
print(cluster_labels)


In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
n_noise_ = list(cluster_labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

### Plotting Results

In [ ]:
def plot_dbscan(X, db):

    # Identify the core samples (ie points that are not outliers)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)

    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=14)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)

    plt.title('Estimated number of clusters: %d' % n_clusters_)
    plt.show()

In [ ]:
plot_dbscan(X, db)

In [ ]:
# Silhouette score
metrics.silhouette_score(X, cluster_labels)

In [ ]:
# Silhouette score only on Core Samples
metrics.silhouette_score(db.components_, cluster_labels[db.core_sample_indices_])

In [ ]:
fig, ax = plt.subplots(10, 10, figsize=(20, 20))
for i, eps in enumerate(np.arange(0.1, 2.0, 0.2)):
    for j, min_samples in enumerate(range(1, 20, 2)):
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        cluster_labels = db.labels_
        n_clusters_ = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        n_noise_ = list(cluster_labels).count(-1)
        print('eps = %.2f, min_samples = %d, Estimated number of clusters: %d' % (eps, min_samples, n_clusters_))

        ax[i, j].scatter(X[:, 0], X[:, 1], c=cluster_labels)
        if i == 0:
            ax[i, j].set_title(f'min_samples={min_samples}')
        if j == 0:
            ax[i, j].set_ylabel(f'eps={eps:.1f}')
plt.tight_layout()
plt.show()

## Moons Dataset

In [ ]:
X, cluster_labels = make_moons(n_samples = 500, noise = 0.1, random_state = 42)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c = cluster_labels)

In [ ]:
# Get Sorted K Nearest Distances
dists = pairwise_distances(X)
dists.sort(axis = 1)
k_dists = dists[:, 3]
k_dists.sort()

In [ ]:
plt.plot(k_dists)

In [ ]:
db = DBSCAN(eps = 0.11, min_samples = 3)
moon_labels = db.fit_predict(X)

In [ ]:
plot_dbscan(X, db)

In [ ]:
# Silhouette score
metrics.silhouette_score(X, moon_labels)

In [ ]:
# Silhouette score only on Core Samples
metrics.silhouette_score(db.components_, moon_labels[db.core_sample_indices_])

In [ ]:
### Circles
X, cluster_labels = make_circles(n_samples = 500, noise = 0.1, factor = 0.1, random_state = 42)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c = cluster_labels)

In [ ]:
# Get Sorted K Nearest Distances
dists = pairwise_distances(X)
dists.sort(axis = 1)
k_dists = dists[:, 3]
k_dists.sort()

In [ ]:
plt.plot(k_dists)

In [ ]:
db = DBSCAN(eps = 0.15, min_samples = 3)
circle_labels = db.fit_predict(X)

In [ ]:
plot_dbscan(X, db)

In [ ]:
# Silhouette score
metrics.silhouette_score(X, circle_labels)

In [ ]:
# Silhouette score only on Core Samples
metrics.silhouette_score(db.components_, circle_labels[db.core_sample_indices_])